Ce projet est un exercice d’Exploratory Data Analysis (EDA).
L’objectif  est de:

    -   Charger le CSV
    -   Faire des stats descriptives + visualisations
    -   Expliquer ce qui influence la décision d’un 2nd date

Le dataset est déjà structuré :

Chaque ligne = un speed-date (une personne qui évalue l’autre)
Nous avons les notes attr (Attractiveness), sinc (Sincerity), intel (Intelligence), fun (Fun), amb (Ambition), shar (Shared interests) (et leurs _o pour l’avis du partenaire)

La décision dec / dec_o / match
Les infos questionnaire (âge, race, revenus, intérêts sportifs, importance donnée à l’attractivité, etc.)
Tout est déjà là.

Le plan d'action est le suivant:
1.Import + chargement
2.Garder seulement les colonnes utiles
3.Nettoyage rapide des données
4.visualiser les données rapidement pour les questions suivantes:
    -   Quels sont les attributs les moins désirables chez un homme / une femme ?
    -   Attractivité perçue vs réelle importance ?
    -   Intérêts partagés > origine ethnique ?
    -   Les gens estiment-ils bien leur valeur sur le marché ?
    -   Mieux d’être le 1er ou le dernier speed-date de la soirée ?

In [5]:
import pandas as pd
import plotly.express as px
import chardet

#on affiche toutes les colonnes
pd.set_option('display.max_columns', None)

#on specifie le fichier a charger
fichier="Speed+Dating+Data.csv"

#on gère les problemes de chargement simplement: check encoding avec la librairie chardet
with open(fichier, 'rb') as f:
    data = f.read()
    # Step 3: Detect Encoding using chardet Library
    encoding_result = chardet.detect(data)
    print(encoding_result)
try:
    df = pd.read_csv(fichier, encoding=encoding_result['encoding'])
    print(f"Fichier '{fichier}' chargé OK")
except:
    raise ValueError(f"probleme lors du chargement du fichier {fichier} , avez vous spécifié le bon encodage? ")

#on regarde la forme du df
print(df.shape)


{'encoding': 'MacRoman', 'confidence': 0.7299985940736158, 'language': ''}
Fichier 'Speed+Dating+Data.csv' chargé OK
(8378, 195)


Objectif 1ere partie (pre-analyse):

Se restreindre dans un 1er temps sur les colonnes qui permettent de:
-   rester focalisé sur ce qui se passe pendant le speed-date (notes + décision) 
       * Les notes pendant l'événement → attr, sinc, … , shar, like, dec, match
       * Les notes reçues → attr_o, dec_o, …
       * Le contexte → gender, order, samerace, wave

-   répondre directement aux 5 questions des « Helpers »:
        * What are the least desirable attributes in a male partner? Does this differ for female partners?
        * How important do people think attractiveness is in potential mate selection vs. its real impact?
        * Are shared interests more important than a shared racial background?
        * Can people accurately predict their own perceived value in the dating market?
        * In terms of getting a second date, is it better to be someone's first speed date of the night or their last?

Ceci afin avoir un notebook clair et pas trop long (15–25 cellules max)

le fichier word annexe fournit la référence pour comprendre toutes les colonnes du fichier CSV. 

Observations:

    codes fréquents dans les colonnes:

        attr:   Attractiveness / Attractive
        sinc:   Sincerity / Sincere
        intel:  Intelligence / Intelligen
        fun:    Fun
        amb:    Ambition / Ambitious
        shar:   Shared interests/hobbies

    aspect temporel:  on identifie des données recoltées:
    
    -   avant le date
    -   juste apres un date
    -   a mi-parcours entres des dates
    -   retrospectivement (après l'evenement)

on identifie les colonnes qui paraissent indispensables d'abord : 

Identifiants & structure : iid, pid, gender, wave, round, order, position
Décision & match : dec, dec_o, match
Notes données par ego : attr, sinc, intel, fun, amb, shar, like, prob
Notes données par alter : attr_o, sinc_o, intel_o, fun_o, amb_o, shar_o
Race & similitude : race, race_o, samerace
Âge : age, age_o

In [7]:
#colonnes identifiées comme indispensable pour demarrer l'EDA
cols_debut = [
    'iid', 'pid', 'gender',          # identité
    'wave', 'round', 'order', 'position',  # contexte du speed_dating
    'age', 'age_o',                  # âge
    'race', 'race_o', 'samerace',    # race
    'attr', 'sinc', 'intel', 'fun', 'amb', 'shar',  # notes données
    'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o',  # notes reçues
    'like', 'prob',                  # like + probabilité estimée
    'dec', 'dec_o', 'match'          # décisions & match
]

-----------------------------------------------------------------------------------------------------------------------------------------

Nous allons identifier les colonnes relatives aux questions posées , par question

Question 1.Least desirable attributes (male vs female) : nous allons verifier dans les colonnes:

    -   gender
    -   attr_o
    -   sinc_o
    -   intel_o
    -   fun_o
    -   amb_o
    -   shar_o
    -   dec_o
    
    les notes données par le genre cible au partenaire (suffixe "_o")

Question 2. declared attractiveness importance vs. its real impact : c'est à dire comparer les attentes déclarées vs notes réelles + décision.
   Les colonnes retenues sont :
   
      attentes déclarées (signup/Time1 "what you look for in the opposite sex"):
          -   attr1_1   rating for expected attractiveness by subject the night of the event

      notes réelles:
         -   attr: 		rating for attractiveness by subject the night of the event, for all 6 attributes
         -   attr_o: 	rating for attractiveness by partner the night of the event, for all 6 attributes
         -   dec: 		decision of subject the night of event
         -   dec_o:        decision of partner the night of event

Question 3.Shared interests vs same racial background:
Shared Interests/Hobbies										

    -   shar:     rating of subject for Shared Interests/Hobbies the night of event
    -   shar_o:     rating of partner for Shared Interests/Hobbies the night of event
    -   int_cor :   correlation between participant’s and partner’s ratings of interests in Time 1
    -   samerace:  	participant and the partner were the same race. 1= yes, 0=no
    -   dec: 		decision of subject the night of event
    -   dec_o:        decision of partner the night of event

Question 4 . Can people predict their own value ?: (Comparer auto-perception vs ce que les autres leur donnent vraiment comme value)

(before the event)
-   expnum: how many do you expect will be interested in dating you? 

(4 mn after the event)
-   prob:     How probable do you think it is that this person will say 'yes' for you?

(the day after the event)
-   attr3_1
-   sinc3_1 … amb3_1 (self-rating)
-   attr5_1 … amb5_1(how others see me)
-    dec_o moyen par iid ((taux de « oui » que les autres ont donné)

Question 5. First vs last date of the night ? (la position dans l'ordre des rencontres speed dating a-t-elle un effet? )
-   order: 		the number of date that night when met partner (nieme partenaire rencontré)
-   dec_o: 		decision of partner the night of event



-----------------------------------------------------------------------------------------------------------------------------------------

Nous allons à present commencer l'inspection des données:

Voici un petit programme Python qui prend une liste de colonnes en entrée et qui génère un rapport (groupement logique + pourcentages + commentaires courts).

In [8]:
def rapport_data_complet(df, colonnes=None, total_lignes=None):
    """
    Rapport complet des valeurs manquantes + suspectes
    Utilise un DataFrame pandas pour un affichage plus propre
    """
    if total_lignes is None:
        total_lignes = len(df)

    if colonnes is None:
        colonnes = df.columns.tolist()

    data = []
    for col in colonnes:
        if col not in df.columns:
            continue

        na_count = df[col].isna().sum()

        # Chaînes problématiques (seulement sur colonnes texte)
        vide = none_str = na_variants = 0
        if df[col].dtype in ['object', 'string']:
            str_col = df[col].astype(str).str.strip().str.lower()
            vide       = (str_col == '').sum()
            none_str   = (str_col == 'none').sum()
            na_variants = str_col.isin(['na', 'n/a', 'null']).sum()

        total_suspect = na_count + vide + none_str + na_variants
        pct = round((total_suspect / total_lignes) * 100, 1) if total_lignes > 0 else 0.0

        # Commentaire court relatif au pct de manques/erronés
        if total_suspect == 0:
            commentaire = "parfait"
        elif pct <= 3:
            commentaire = "excellent"
        elif pct <= 6:
            commentaire = "très bon"
        elif pct <= 10:
            commentaire = "bon"
        elif pct <= 15:
            commentaire = "moyen – prudence"
        else:
            commentaire = "problématique"

        data.append({
            'Colonne': col,
            'NaN': na_count,
            'Vide': vide,
            'None': none_str,
            'na/n/a/null': na_variants,
            'Total suspect': total_suspect,
            '%': pct,
            'Commentaire': commentaire
        })

    # Création du DataFrame
    rapport_df = pd.DataFrame(data)

    # Tri : parfait d'abord, puis par % croissant
    rapport_df = rapport_df.sort_values(by=['Total suspect', '%'])

    # ────────────────────────────────────────────────
    # Affichage
    # ────────────────────────────────────────────────
    print(f"\n=== Rapport complet – {total_lignes:,} lignes ===\n")

    # 1. Colonnes parfaites
    parfaites = rapport_df[rapport_df['Total suspect'] == 0]['Colonne'].tolist()
    if parfaites:
        print("1. Colonnes sans aucun problème (0 suspect)")
        print("On peut les utiliser sans filtre :")
        print(", ".join(parfaites))
        print()

    # Tableau principal avec Styler (optionnel : couleurs)
    print("2. Rapport détaillé (toutes colonnes analysées)")

    print(rapport_df.to_markdown(index=False, tablefmt="pipe", floatfmt=".1f"))   # ← très important : affiche le tableau stylé dans Jupyter/Colab

    # Note + recommandations génériques
    print("\nNote : les colonnes numériques convertissent souvent '' ou 'None' en NaN lors du chargement.")
    print("Pour les colonnes texte, les chaînes vides et 'na'/'None' sont plus fréquentes.\n")
#guidelines relatives aux manques et leur impact
    print("""
            Recommandations pratiques génériques (indépendant des colonnes analysées) :
            • Colonnes avec 0 % ou < 3 % de manquants
                → Utilisation sans restriction (moyennes, corrélations, regroupements, visualisations)
            • Colonnes avec 3 % à 6–7 % de manquants
                → Très souvent acceptable pour des analyses descriptives
                → On peut calculer des moyennes, médianes, distributions sans filtre
                → Pour des corrélations ou modèles → filtrer les lignes incomplètes si possible
            • Colonnes avec 8 % à 15 % de manquants
                → Utilisation avec prudence
                → Préférer filtrer les lignes incomplètes (dropna) quand cette colonne est centrale
                → Attention au biais potentiel si les manquants ne sont pas aléatoires
            • Colonnes > 15 % de manquants
                → À manipuler très prudemment
                → Déconseillé pour des moyennes / corrélations globales sans filtre
                → À réserver à des analyses secondaires ou à des sous-populations complètes
                → Éviter de construire des scores / moyennes qui incluent fortement cette colonne

            Astuces opérationnelles :
            • Pour toute analyse impliquant plusieurs colonnes → utiliser dropna(subset=[...]) sur les variables clés
            • Perte < 5–6 % → généralement négligeable sur 8000+ lignes
            • Perte > 10–12 % → toujours vérifier le nombre de lignes restantes avant / après filtre
        """)

# ────────────────────────────────────────────────
# Exemple d'appel
# ────────────────────────────────────────────────
# Appel des colonnes identifiées au debut du projet
rapport_data_complet(df, colonnes=cols_debut)


=== Rapport complet – 8,378 lignes ===

1. Colonnes sans aucun problème (0 suspect)
On peut les utiliser sans filtre :
iid, gender, wave, round, order, position, samerace, dec, dec_o, match

2. Rapport détaillé (toutes colonnes analysées)
| Colonne   |   NaN |   Vide |   None |   na/n/a/null |   Total suspect |    % | Commentaire      |
|:----------|------:|-------:|-------:|--------------:|----------------:|-----:|:-----------------|
| iid       |     0 |      0 |      0 |             0 |               0 |  0.0 | parfait          |
| gender    |     0 |      0 |      0 |             0 |               0 |  0.0 | parfait          |
| wave      |     0 |      0 |      0 |             0 |               0 |  0.0 | parfait          |
| round     |     0 |      0 |      0 |             0 |               0 |  0.0 | parfait          |
| order     |     0 |      0 |      0 |             0 |               0 |  0.0 | parfait          |
| position  |     0 |      0 |      0 |             0 |     

Nous effectuons quelques vérifications complémentaires sur les colonnes

## on note qu'il y a beaucoup de données manquantes. il faudra adapter la manière dont on gère les manques suivant les questions auxquelles on veut repondre.

In [9]:
print("Nombre de lignes et colonnes :",df.shape )  

# iid: 	unique subject number, group(wave id gender)
nb_part=df["iid"].nunique()
print("Nombre de participants uniques :",nb_part)  

#on verifie le nombres correct des data importantes 
nb_speed_dates=df.shape[0]
print("Nombre de speed-dates :", nb_speed_dates)  


Nombre de lignes et colonnes : (8378, 195)
Nombre de participants uniques : 551
Nombre de speed-dates : 8378


#Vérifier les Statistiques et valeurs manquantes notes (attr, sinc, etc.)

In [10]:
notes_donnees=['attr', 'sinc', 'intel', 'fun', 'amb', 'shar']
notes_recues=['attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o']
df[notes_donnees].describe()


,attr,sinc,intel,fun,amb,shar
count,8176.000000,8101.000000,8082.000000,8028.000000,7666.000000,7311.000000
mean,6.189995,7.175164,7.368597,6.400598,6.777524,5.474559
std,1.950169,1.740315,1.550453,1.953702,1.794055,2.156363
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,6.000000,6.000000,5.000000,6.000000,4.000000
50%,6.000000,7.000000,7.000000,7.000000,7.000000,6.000000
75%,8.000000,8.000000,8.000000,8.000000,8.000000,7.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


En regardant au passage les STD , on note ici que l'intelligence et la sincérité sont les attributs les moins discriminants → la plupart des participants reçoivent des notes élevées et proches (médiane 7, peu de notes < 5).
L'attractivité et surtout les intérêts partagés sont beaucoup plus polarisants → c'est là qu'on voit les plus gros écarts de jugement.
Cela reflète bien la réalité du dating : on pardonne plus facilement un manque d'ambition ou de fun qu'un manque d'attractivité ou d'intérêts communs.

In [11]:
data = {
    'Critère': ['Attractivité', 'Sincérité', 'Intelligence', 'Fun', 'Ambition', 'Intérêts communs'],
    'Moyenne': [6.19, 7.18, 7.37, 6.40, 6.78, 5.47],
    'Écart-type': [1.95, 1.74, 1.55, 1.95, 1.79, 2.16]
}

df2 = pd.DataFrame(data)

# Version barre avec barres d'erreur (la plus claire et rapide)
fig = px.bar(
    df2,
    x='Critère',
    y='Moyenne',
    error_y='Écart-type',
    title='Notes moyennes attribuées en speed-dating (± écart-type)',
    labels={'Moyenne': 'Note moyenne (sur 10)'},
    color='Critère',
    text_auto='.2f'
)

fig.update_traces(textposition='outside', cliponaxis=False)
fig.update_layout(
    yaxis_range=[0, 10],
    xaxis_title="",
    showlegend=False,
    height=520,
    bargap=0.25,
    template='plotly_white'
)

fig.show()

In [12]:
import pandas as pd
import plotly.express as px

# Format long
df_long = df[['attr', 'sinc', 'intel', 'fun', 'amb', 'shar']].melt(
    var_name='Critère_original',
    value_name='Note'
)

mapping = {
    'attr': 'Attractivité',
    'sinc': 'Sincérité',
    'intel': 'Intelligence',
    'fun': 'Fun',
    'amb': 'Ambition',
    'shar': 'Intérêts communs'
}
df_long['Critère'] = df_long['Critère_original'].map(mapping)

fig = px.violin(
    df_long,
    x='Note',
    y='Critère',
    color='Critère',
    orientation='h',
    box=True,                    # ajoute la boîte + médiane
    points='outliers',           # montre les rares extrêmes
    title='Distribution des notes attribuées (violin + boîte + outliers)',
    template='plotly_white'
)

fig.update_traces(
    meanline_visible=True,       # ligne de moyenne visible
    side='positive',
    spanmode='soft'
)

fig.update_layout(
    xaxis_range=[0, 10],
    xaxis_title='Note (sur 10)',
    yaxis_title='',
    showlegend=False,
    height=700,
    width=1000,
    violingap=0.3,
    violinmode='overlay'
)

fig.show()

In [13]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Format long
df_long = df[['attr', 'sinc', 'intel', 'fun', 'amb', 'shar']].melt(
    var_name='Critère_original',
    value_name='Note'
)

mapping = {
    'attr': 'Attractivité',
    'sinc': 'Sincérité',
    'intel': 'Intelligence',
    'fun': 'Fun',
    'amb': 'Ambition',
    'shar': 'Intérêts communs'
}
df_long['Critère'] = df_long['Critère_original'].map(mapping)

# Calcul moyenne par critère pour annotation
moyennes = df_long.groupby('Critère')['Note'].mean().round(2)

fig = px.box(
    df_long,
    x='Note',
    y='Critère',
    color='Critère',
    orientation='h',
    points='outliers',               # ou 'all' 
    title='Distribution des notes attribuées (boîte + points + moyenne)',
)

# Ajout des moyennes comme annotation
for critere, moy in moyennes.items():
    fig.add_annotation(
        x=moy,
        y=critere,
        text=f"<b>moy = {moy}</b>",
        showarrow=True,
        arrowhead=2,
        ax=40,
        ay=-30,
        font=dict(size=12, color="black"),
        bgcolor="white",
        bordercolor="black",
        borderwidth=1
    )

fig.update_traces(
    boxpoints='all',
    jitter=0.4,
    pointpos=-0.2,
    marker=dict(size=5, opacity=0.7)
)

fig.update_layout(
    xaxis_range=[0, 10],
    xaxis_title='Note (sur 10)',
    yaxis_title='',
    showlegend=False,
    height=700,
    width=1000,
    template='plotly_white'
)

fig.show()

on regarde les notes les plus faibles et les plus hautes, ainsi que leur ratio

In [14]:
import pandas as pd

notes_cols = ['attr', 'sinc', 'intel', 'fun', 'amb', 'shar']
low_threshold = 3
high_threshold = 8

results = []
for col in notes_cols:
    total = df[col].notna().sum()
    if total == 0:
        continue
    
    low_count = (df[col] <= low_threshold).sum()
    high_count = (df[col] >= high_threshold).sum()
    
    results.append({
        'Critère': col,
        'Total notes valides': total,
        f'% faibles (≤ {low_threshold})': round(low_count / total * 100, 1),
        f'% hautes (≥ {high_threshold})': round(high_count / total * 100, 1),
        'Ratio hautes / faibles': round(high_count / low_count, 1) if low_count > 0 else '∞'
    })

df_extremes = pd.DataFrame(results)

# Affichage amélioré (sans index, colonnes alignées)
# Juste un print propre
df_extremes = df_extremes.round(1)
df_extremes['Total notes valides'] = df_extremes['Total notes valides'].astype(int)
print(df_extremes.to_string(index=False))

Critère  Total notes valides  % faibles (≤ 3)  % hautes (≥ 8)  Ratio hautes / faibles
   attr                 8176              9.2            25.7                     2.8
   sinc                 8101              3.2            45.8                    14.5
  intel                 8082              1.5            49.3                    32.6
    fun                 8028              7.7            30.5                     3.9
    amb                 7666              4.2            36.1                     8.6
   shar                 7311             18.8            17.6                     0.9


# QUESTION 1 : Least desirable attributes (male vs female) :

## colonnes retenues:
-   gender
-   attr_o
-   sinc_o
-   intel_o
-   fun_o
-   amb_o
-   shar_o
-   dec_o

*les notes données par le genre cible au partenaire (suffixe "_o")*

## nous allons calculer la moyenne

In [15]:
liste_criteres=['attr_o','sinc_o','intel_o','fun_o','amb_o','shar_o','dec_o']
df[liste_criteres].mean()

attr_o     6.190411
sinc_o     7.175256
intel_o    7.369301
fun_o      6.400599
amb_o      6.778409
shar_o     5.474870
dec_o      0.419551
dtype: float64

In [ ]:
px.pie(

<function plotly.express._chart_types.pie(data_frame=None, names=None, values=None, color=None, facet_row=None, facet_col=None, facet_col_wrap=0, facet_row_spacing=None, facet_col_spacing=None, color_discrete_sequence=None, color_discrete_map=None, hover_name=None, hover_data=None, custom_data=None, category_orders=None, labels=None, title=None, subtitle=None, template=None, width=None, height=None, opacity=None, hole=None) -> plotly.graph_objs._figure.Figure>